In [6]:
import geopandas as gpd
import pandas as pd
import fiona
import numpy as np
from pyproj import CRS
from shapely.geometry import Point
from shapely.geometry import shape
pd.set_option('display.max_columns', None)

# Setting up the corrspondance matrix of shapefiles

In [7]:
ref_path= '../Data/Mapping layout/Admin3/uga_admbnda_adm3_UBOS_v5_cleaned [CB].shp'
poverty_path = "../Data/Poverty/Subcounty/SubCountyShapefiles [from GIS WG]/Uganda2018SubcountyMultidimensionalPovertyConsensual.shp"

In [8]:
ref_shapes = gpd.read_file(ref_path)
poverty_shapes = gpd.read_file(poverty_path)


In [9]:
poverty_shapes.head()

,SP_ID,FID_1,OBJECTID,District,Dcode,County,CCode,Subcounty,SCCode,pov_count,pov_per,ChildPov,scode,SCODE_num,Sccode_1,HH_Poverty,Child_Pov,SCProp,HHImprov,ChpovImp,ANC4,F15Regions,DName2016,DNama2017,dc2017,DName2018,dc2018,DName2019,DistProp,HouseholdC,subreg,district_c,Individual,ChildConsP,AdultConsP,geometry
0,0,337,58,SHEEMA,426,SHEEMA,4261,BUGONGI TOWN COUNCIL,426101,0.084,8.435,0.092,426101,426101,426101.0,0.063365,0.0734,Bugongi town council,0.325653,0.253406,0.0,ANKOLE,SHEEMA,SHEEMA,426.0,SHEEMA,426.0,SHEEMA,Sheema,78-94,10.0,330.0,82-95,84-95,79-94,"POLYGON ((30.28780 -0.61772, 30.28820 -0.62063..."
1,1,338,59,SHEEMA,426,SHEEMA,4261,KIGARAMA,426103,0.076,7.641,0.083,426103,426103,426103.0,0.063771,0.0737,Kigarama,0.191764,0.126187,0.0,ANKOLE,SHEEMA,SHEEMA,426.0,SHEEMA,426.0,SHEEMA,Sheema,78-94,10.0,330.0,82-95,84-95,79-94,"POLYGON ((30.46856 -0.49585, 30.46835 -0.49833..."
2,2,339,60,SHEEMA,426,SHEEMA,4261,KITAGATA,426104,0.079,7.923,0.087,426104,426104,426104.0,0.052619,0.0615,Kitagata,0.501359,0.414634,0.0,ANKOLE,SHEEMA,SHEEMA,426.0,SHEEMA,426.0,SHEEMA,Sheema,78-94,10.0,330.0,82-95,84-95,79-94,"POLYGON ((30.16429 -0.66719, 30.16432 -0.66600..."
3,3,340,61,SHEEMA,426,SHEEMA,4261,KAKINDO TOWN COUNCIL,426110,0.074,7.446,0.081,426110,426110,426110.0,0.057468,0.0671,Kakindo town council,0.287673,0.207153,0.0,ANKOLE,SHEEMA,SHEEMA,426.0,SHEEMA,426.0,SHEEMA,Sheema,78-94,10.0,330.0,82-95,84-95,79-94,"POLYGON ((30.36533 -0.49214, 30.36511 -0.49346..."
4,4,341,62,SHEEMA,426,SHEEMA,4261,MASHERUKA,426106,0.080,7.994,0.087,426106,426106,426106.0,0.055703,0.0646,Masheruka,0.436188,0.346749,0.0,ANKOLE,SHEEMA,SHEEMA,426.0,SHEEMA,426.0,SHEEMA,Sheema,78-94,10.0,313.0,82-95,84-95,79-94,"POLYGON ((30.43425 -0.47008, 30.43520 -0.46803..."


In [10]:
# Checking I can anchor on ADM3_PCODE and SCCode

print(
    len(ref_shapes['ADM3_PCODE']),
    len(set(ref_shapes['ADM3_PCODE'])),
    len(poverty_shapes['SCCode']),
    len(set(poverty_shapes['SCCode'])))

# Turns out I need to combine name and code for the poverty data 

poverty_shapes['concat']=poverty_shapes['Subcounty']+poverty_shapes['SCCode']



1438 1438 1517 1485


In [11]:
# Checking CRS

print(ref_shapes.crs)
print(poverty_shapes.crs)

# Converting CRS

poverty_shapes.crs="epsg:4326"
print(poverty_shapes.crs)

epsg:4326
None
epsg:4326


In [12]:
# Change indexes 
ref_shapes.set_index('ADM3_PCODE',inplace=True, drop=True)
poverty_shapes.set_index('concat',drop=True, inplace=True)

# Extract only polygons
ref_geom = ref_shapes["geometry"].centroid
poverty_geom = poverty_shapes["geometry"].centroid

In [13]:
# from a first run f codes, two polygons are not valid from the poverty data, So lets drop them 
invalid_shapes=['MIDIGO313110','NYIMBWA104207']
poverty_shapes.drop(invalid_shapes,axis=0,inplace=True)
poverty_geom.drop(invalid_shapes,axis=0,inplace=True)

In [14]:
%%time

convert=pd.DataFrame()

for i in list(poverty_shapes.index):
    for j in list(ref_shapes.index):
        x = ref_geom.loc[j].distance(poverty_geom.loc[i])
        convert.loc[j,i]=x

Wall time: 15min 18s


In [15]:
dict_pov_to_ref={}
for x in list(convert.columns):
    y = ref_shapes.index[np.argmin(convert[x])]
    dict_pov_to_ref[x]=y

In [16]:
# Check totals and a speficic subcounty area are around the same

print(ref_geom.loc['UG314101'].distance(poverty_geom.loc['ABIM314101']))

7.714625043870724e-06


In [ ]:
poverty_map=poverty_shapes.copy().drop('geometry',axis=1)
poverty_map.reset_index(inplace=True)
poverty_map['ADM3_PCODE']=poverty_map['concat'].replace(dict_pov_to_ref)
poverty_map.set_index('ADM3_PCODE',drop=True,inplace=True)
#poverty_map.to_csv('../Data/Results/poverty_map.csv')

In [ ]:
poverty_map.head()